这篇论文主要通过**自然语言处理（NLP）和社会网络分析SNA**的方法，对《三体》系列小说中的人物关系进行量化研究，并结合女性主义理论进行解读。分析的核心包括以下几个方面：

# **1.数据分析方法概述**
### **（1）文本数据预处理**
1. **深度阅读文本，构建人物词典**
   - 先手工整理人物名单，建立两份词典：
     - **正式姓名词典**（如：罗辑、程心、叶文洁）
     - **别称映射词典**（如：罗辑 -> 小罗、罗兄、罗教授）

2. **实体识别**
   - 使用**Python的NLP工具（如jieba、spaCy等）**进行**分词**和**实体识别**，提取小说中的人物名称。

---

### **（2）人物关系构建**
1. **共现分析（Co-occurrence Analysis）**
   - **定义人物关系：** 两个人物在小说**同一行**同时出现，视为存在一次联系。
   - **建立共现矩阵：** 统计小说文本中人物的共现次数，形成**人物-人物共现矩阵**（即邻接矩阵）。

2. **数据清洗**
   - 过滤出现次数较少的人物，最终确定**180名人物**和**368对共现关系**。

---

### **（3）社会网络分析**
1. **使用 Gephi 进行可视化**
   - 导入清洗后的**人物-人物共现矩阵**，构建社交网络。
   - **节点（Node）：** 代表小说中的角色。
   - **边（Edge）：** 代表人物间的联系（共现次数）。
   - 过滤掉**共现次数小于5的弱联系**，确保主要人物关系清晰可见。

2. **计算社会网络中心性指标**
   - **度中心性（Degree Centrality）**
     - 计算角色的**直接连接数**，代表人物的影响力。
     - 统计**度中心性最高的角色**，发现**智子、程心**等女性角色占据重要位置。

   - **中介中心性（Betweenness Centrality）**
     - 计算角色**在其他人物间的最短路径上**出现的次数，衡量其在网络中的“桥梁”作用。
     - 发现**智子在整个社交网络中是最核心的节点**，连接多个主要角色。

   - **特征向量中心性（Eigenvector Centrality）**
     - 计算角色是否与其他**重要角色**有较多联系，衡量人物的社会影响力。
     - 发现**智子、程心、叶文洁等女性角色在小说中具有很强的网络影响力**。

# **2.实验流程**
## **1. 数据准备**
### **（1）所需数据**
- **小说文本**：《三体》全书的文本数据（TXT格式）。
- **人物词典**：
  - **正式姓名词典**（如：罗辑、程心、叶文洁）
  - **别称映射词典**（如：罗辑 -> 小罗、罗教授）
- **共现矩阵**：
  - **记录人物在文本中的共现次数**，即两个角色同时出现在**相同行**中。


In [9]:
import jieba
import jieba.posseg as pseg
import pandas as pd
import networkx as nx
import itertools
from collections import Counter

In [10]:
# 读取小说文本
with open("san_ti.txt", "r", encoding="utf-8") as f:
    text = f.readlines()

# 读取正式姓名词典
with open("formal_names.txt", "r", encoding="utf-8") as f:
    formal_names = [line.strip() for line in f.readlines()]

# 读取别名映射词典
alias_mapping = {}
with open("alias_mapping.txt", "r", encoding="utf-8") as f:
    for line in f:
        main_name, aliases = line.strip().split(":")
        for alias in aliases.split(","):
            alias_mapping[alias.strip()] = main_name.strip()

### **（2）数据来源**
1. **小说文本数据**
   - **方式1（手动）：** 可以从电子书（如 Kindle、网盘资源）导出《三体》文本。
   - **方式2（爬取）：** 可使用 Python 爬取在线小说网站（如起点、豆瓣读书）。
   - **方式3（公开数据集）：** 查找 GitHub 或 NLP 公开数据资源。

2. **人物词典**
   - **手工整理**：《三体》中的人物可以手动提取，构建正式姓名词典。
   - **自动生成**：
     - 从小说文本中提取所有**高频人名**。
     - 结合 NLP **共现分析**自动发现别称。

3. **共现矩阵**
   - **手工标注**：找出**主要角色的共现关系**，建立基础数据。
   - **自动生成**：
     - 使用 NLP 分词后，检测**人物名**是否在**同一行或相邻N行**出现。
     - 统计**人物对Character Pairs**的共现次数。


## **2. 数据预处理**
### **（1）分词与实体识别**
**目标：**
- 使用 **jieba** 或 **HanLP** 进行**中文分词**，识别**人名**。
- 过滤掉**非人名的词**，形成**正式姓名列表**。
- （分词与别名替换）


In [11]:
# 加载人物词典到 jieba
jieba.load_userdict("formal_names.txt")

def replace_aliases(text, alias_mapping):
    """ 替换文本中的别名为正式姓名 """
    for alias, main_name in alias_mapping.items():
        text = text.replace(alias, main_name)
    return text

# 替换小说文本中的别名
processed_text = [replace_aliases(line, alias_mapping) for line in text]

# 统计人名出现频率
char_counts = Counter()
for line in processed_text:
    words = pseg.cut(line)
    for word, flag in words:
        if flag == "nr" and word in formal_names:
            char_counts[word] += 1

### **（2）共现矩阵构建**
**目标：**
- 计算**人物之间的共现次数**。
- 统计两个人物**在同一行或相邻N行**出现的频率。

In [12]:
window_size = 3  # 设置窗口大小
co_occurrence = {char: {char2: 0 for char2 in formal_names} for char in formal_names}

for i in range(len(processed_text)):
    window_text = " ".join(processed_text[i:i + window_size])
    present_chars = [char for char in formal_names if char in window_text]
    
    for char1, char2 in itertools.combinations(present_chars, 2):
        co_occurrence[char1][char2] += 1
        co_occurrence[char2][char1] += 1

# 转换为 DataFrame 并保存
df_co_occurrence = pd.DataFrame(co_occurrence)
df_co_occurrence.to_csv("co_occurrence_matrix.csv")

## **3. 社会网络分析**
### **（1）使用 NetworkX 构建社交网络**
**目标：**
- 用 **NetworkX** 构建人物关系图。
- 计算 **社交网络中心性指标**（度中心性、中介中心性、特征向量中心性）。

In [13]:
G = nx.Graph()

for character in df_co_occurrence.columns:
    G.add_node(character, weight=df_co_occurrence[character].sum())

for char1 in df_co_occurrence.columns:
    for char2 in df_co_occurrence.columns:
        weight = df_co_occurrence.loc[char1, char2]
        if weight > 5:  # 过滤低频共现
            G.add_edge(char1, char2, weight=weight)

### **（2）计算中心性指标**

In [14]:
# 计算中心性指标
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)

## **4. Gephi 可视化**
### **（1）导出数据到 Gephi**
- **Gephi** 是一个专业的数据可视化工具，可以加载 `.gexf` 文件，绘制人物关系图。
- 通过调整**颜色、节点大小、边的权重**，可以直观分析**《三体》中的人物关系网**。

In [15]:
nx.write_gexf(G, "san_ti_network.gexf")

In [16]:
# 输出最高中心性角色
centrality_df = pd.DataFrame({
    "Character": list(degree_centrality.keys()),
    "Degree": list(degree_centrality.values()),
    "Betweenness": list(betweenness_centrality.values()),
    "Eigenvector": list(eigenvector_centrality.values())
})
print(centrality_df.sort_values(by="Degree", ascending=False).head(10))

    Character    Degree  Betweenness  Eigenvector
44         罗辑  0.232394     0.117930     0.348824
40      军事执政官  0.218310     0.144006     0.277735
116        智子  0.197183     0.066246     0.333098
15        叶文洁  0.154930     0.095421     0.216419
97         程心  0.147887     0.050368     0.229816
36       核弹女孩  0.133803     0.040541     0.270506
1          史强  0.119718     0.044090     0.186774
0          汪淼  0.112676     0.029070     0.139902
101       云天明  0.105634     0.028736     0.142989
46   弗雷德里克·泰勒  0.098592     0.018579     0.195734


---
## **5.实验流程总结**
✅ **数据准备**
- 获取《三体》文本数据（TXT）。
- 构建**人物词典**（正式姓名+别称映射）。
- 统计**人物共现矩阵**。

✅ **数据处理**
- **分词+实体识别**（使用 NLP 提取人名）。
- **统计共现次数**（计算两两角色的关系）。

✅ **网络分析**
- 使用 **NetworkX** 计算**度中心性、中介中心性、特征向量中心性**。
- 发现《三体》中**智子、程心、叶文洁**等女性角色的核心地位。

✅ **Gephi 可视化**
- 生成 `.gexf` 文件，在 Gephi 中绘制社交网络。

通过本实验，可以定量研究《三体》中的**人物社交网络**，并结合**女性主义理论**进行深入解读。
# **3. 论文的核心观点总结**
1. **人物网络分析方法**
   - 通过NLP提取人物，并构建**共现矩阵**。
   - 通过Python和Gephi构建**社交网络**并分析中心性。

2. **女性角色的重要性**
   - **智子、程心、叶文洁**等女性角色在**度中心性、中介中心性、特征向量中心性**等指标上排名前列。
   - 说明她们在故事中的社交网络中占据**重要的资源和权力位置**。

3. **女性主义视角**
   - 论文通过波伏娃的女性主义理论，认为《三体》的女性角色成功突破了**传统父权社会的束缚**。
   - 小说中的未来社会呈现**“女性化”趋势**，女性在社会网络中获得了更大的**话语权和影响力**。

---


